In [1]:
import torch

import random
import numpy as np

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [3]:
len(tokenizer.vocab)

30522

In [4]:
tokens = tokenizer.tokenize('Hello WORLD how ARE yoU?')

print(tokens)

['hello', 'world', 'how', 'are', 'you', '?']


In [5]:
indexes = tokenizer.convert_tokens_to_ids(tokens)

print(indexes)

[7592, 2088, 2129, 2024, 2017, 1029]


In [6]:
init_token = tokenizer.cls_token
eos_token = tokenizer.sep_token
pad_token = tokenizer.pad_token
unk_token = tokenizer.unk_token

print(init_token, eos_token, pad_token, unk_token)

[CLS] [SEP] [PAD] [UNK]


In [7]:
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

print(init_token_idx, eos_token_idx, pad_token_idx, unk_token_idx)

101 102 0 100


In [8]:
max_input_length = tokenizer.max_model_input_sizes['bert-base-uncased']

print(max_input_length)

512


In [9]:
def tokenize_and_cut(sentence):
    tokens = tokenizer.tokenize(sentence) 
    tokens = tokens[:max_input_length-2]
    return tokens

In [10]:
from torchtext import data

TEXT = data.Field(batch_first = True,
                  use_vocab = False,
                  tokenize = tokenize_and_cut,
                  preprocessing = tokenizer.convert_tokens_to_ids,
                  init_token = init_token_idx,
                  eos_token = eos_token_idx,
                  pad_token = pad_token_idx,
                  unk_token = unk_token_idx)

LABEL = data.LabelField(dtype = torch.float)

In [11]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

train_data, valid_data = train_data.split(random_state = random.seed(SEED))

In [12]:
print(f"Number of training examples: {len(train_data)}")
print(f"Number of validation examples: {len(valid_data)}")
print(f"Number of testing examples: {len(test_data)}")

Number of training examples: 17500
Number of validation examples: 7500
Number of testing examples: 25000


In [13]:
print(vars(train_data.examples[6]))

{'text': [1996, 1000, 8785, 1000, 7814, 2000, 2023, 2003, 6414, 1037, 21025, 7382, 6799, 2000, 3046, 2000, 2275, 2023, 2694, 2265, 4237, 2013, 1996, 8817, 1997, 2060, 8872, 3065, 1012, 1996, 2069, 2417, 21564, 2075, 7814, 2000, 2023, 2265, 2003, 6487, 19084, 1010, 2348, 2010, 2476, 2442, 2031, 2042, 1006, 6151, 6810, 2099, 7178, 2135, 1007, 14071, 2075, 2044, 2642, 7524, 2065, 2002, 2772, 2039, 2005, 2023, 8040, 7317, 7432, 1012, 1026, 7987, 1013, 1028, 1026, 7987, 1013, 1028, 1996, 20342, 1011, 23384, 1997, 1996, 1000, 8785, 1000, 7814, 2000, 1996, 2265, 2003, 4372, 17695, 23722, 4383, 1999, 2028, 2792, 2522, 1011, 4626, 10223, 6323, 8109, 1006, 2029, 2074, 23283, 2008, 2023, 2265, 2003, 1996, 2197, 9277, 1997, 1996, 9636, 1012, 1007, 1999, 2344, 2000, 4608, 1037, 21329, 1010, 1996, 1000, 13235, 1000, 3594, 2070, 3399, 2055, 1000, 17255, 1000, 1012, 2061, 1010, 2002, 3957, 2023, 2146, 7526, 2008, 1010, 2065, 2057, 2031, 2464, 1996, 8343, 1999, 3182, 1037, 1010, 1038, 1998, 1039, 1010,

In [14]:
tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[6])['text'])

print(tokens)

['the', '"', 'math', '"', 'aspect', 'to', 'this', 'is', 'merely', 'a', 'gi', '##mm', '##ick', 'to', 'try', 'to', 'set', 'this', 'tv', 'show', 'apart', 'from', 'the', 'millions', 'of', 'other', 'cop', 'shows', '.', 'the', 'only', 'red', '##eem', '##ing', 'aspect', 'to', 'this', 'show', 'is', 'rob', 'morrow', ',', 'although', 'his', 'career', 'must', 'have', 'been', '(', 'und', '##ese', '##r', '##ved', '##ly', ')', 'wan', '##ing', 'after', 'northern', 'exposure', 'if', 'he', 'signed', 'up', 'for', 'this', 'sc', '##hl', '##ock', '.', '<', 'br', '/', '>', '<', 'br', '/', '>', 'the', 'lame', '-', 'ness', 'of', 'the', '"', 'math', '"', 'aspect', 'to', 'the', 'show', 'is', 'en', '##cap', '##sul', '##ated', 'in', 'one', 'episode', 'co', '-', 'starring', 'lou', 'diamond', 'phillips', '(', 'which', 'just', 'confirms', 'that', 'this', 'show', 'is', 'the', 'last', 'refuge', 'of', 'the', 'damned', '.', ')', 'in', 'order', 'to', 'catch', 'a', 'fugitive', ',', 'the', '"', 'mathematician', '"', 'uses'

In [15]:
LABEL.build_vocab(train_data)

In [16]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


In [17]:
BATCH_SIZE = 128

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

In [18]:
from transformers import BertTokenizer, BertModel

bert = BertModel.from_pretrained('bert-base-uncased')

In [19]:
import torch.nn as nn

class BERTGRUSentiment(nn.Module):
    def __init__(self,
                 bert,
                 hidden_dim,
                 output_dim,
                 n_layers,
                 bidirectional,
                 dropout):
        
        super().__init__()
        
        self.bert = bert
        
        embedding_dim = bert.config.to_dict()['hidden_size']
        
        self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
        
        self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [batch size, sent len]
                
        with torch.no_grad():
            embedded = self.bert(text)[0]
                
        #embedded = [batch size, sent len, emb dim]
        
        _, hidden = self.rnn(embedded)
        
        #hidden = [n layers * n directions, batch size, emb dim]
        
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
                
        #hidden = [batch size, hid dim]
        
        output = self.out(hidden)
        
        #output = [batch size, out dim]
        
        return output

In [20]:
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = BERTGRUSentiment(bert,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)

In [21]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 112,241,409 trainable parameters


In [22]:
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False

In [23]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,759,169 trainable parameters


In [24]:
for name, param in model.named_parameters():                
    if param.requires_grad:
        print(name)

rnn.weight_ih_l0
rnn.weight_hh_l0
rnn.bias_ih_l0
rnn.bias_hh_l0
rnn.weight_ih_l0_reverse
rnn.weight_hh_l0_reverse
rnn.bias_ih_l0_reverse
rnn.bias_hh_l0_reverse
rnn.weight_ih_l1
rnn.weight_hh_l1
rnn.bias_ih_l1
rnn.bias_hh_l1
rnn.weight_ih_l1_reverse
rnn.weight_hh_l1_reverse
rnn.bias_ih_l1_reverse
rnn.bias_hh_l1_reverse
out.weight
out.bias


In [25]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [26]:
criterion = nn.BCEWithLogitsLoss()

In [27]:
model = model.to(device)
criterion = criterion.to(device)

In [28]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [29]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [30]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [31]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [32]:
N_EPOCHS = 20

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
        
    end_time = time.time()
        
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
        
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut6-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 10m 53s
	Train Loss: 0.483 | Train Acc: 75.83%
	 Val. Loss: 0.274 |  Val. Acc: 89.07%
Epoch: 02 | Epoch Time: 10m 45s
	Train Loss: 0.280 | Train Acc: 88.62%
	 Val. Loss: 0.236 |  Val. Acc: 90.43%
Epoch: 03 | Epoch Time: 10m 46s
	Train Loss: 0.232 | Train Acc: 90.73%
	 Val. Loss: 0.282 |  Val. Acc: 88.83%
Epoch: 04 | Epoch Time: 10m 45s
	Train Loss: 0.206 | Train Acc: 92.20%
	 Val. Loss: 0.227 |  Val. Acc: 90.83%
Epoch: 05 | Epoch Time: 10m 45s
	Train Loss: 0.182 | Train Acc: 93.01%
	 Val. Loss: 0.253 |  Val. Acc: 90.18%
Epoch: 06 | Epoch Time: 10m 45s
	Train Loss: 0.156 | Train Acc: 94.13%
	 Val. Loss: 0.241 |  Val. Acc: 91.07%
Epoch: 07 | Epoch Time: 10m 46s
	Train Loss: 0.132 | Train Acc: 95.14%
	 Val. Loss: 0.248 |  Val. Acc: 91.14%
Epoch: 08 | Epoch Time: 10m 45s
	Train Loss: 0.108 | Train Acc: 96.09%
	 Val. Loss: 0.267 |  Val. Acc: 91.02%


KeyboardInterrupt: 

In [33]:
model.load_state_dict(torch.load('tut6-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.216 | Test Acc: 91.28%
